In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

from os import listdir
from os.path import isfile, join

In [4]:
deg_df = pd.read_csv('data/SS/92_DEG_Clusters.csv', index_col=0)
deg_genes = deg_df.index

In [3]:
raw_file_dir = 'data/GSE111062_RAW/'
onlyfiles = [f for f in listdir(raw_file_dir) if isfile(join(raw_file_dir, f))]

df_list = []
for raw_file in onlyfiles:
    raw_file_path = raw_file_dir + raw_file
    series_name = raw_file.split('.txt')[0].split('_')[1]
    df = pd.read_csv(raw_file_path, compression='gzip', sep='\t', index_col=0, names=[series_name])
    df_list.append(df)

df = pd.concat(df_list, axis=1)
df.T.to_csv('data/GSE111062_RAW/all.csv')

from operator import index


normalized_df = pd.read_csv('All_samples_RCPM_table.csv', index_col=0)
# normalized_df.T.std()

r = re.compile("GL.*")
newlist = list(filter(r.match, df.columns)) # Read Note below
gl_series = sorted(newlist, key=lambda x: float(x.split('h')[0].split('GL')[1]))
r = re.compile("HL.*")
newlist = list(filter(r.match, df.columns)) # Read Note below
hl_series = sorted(newlist, key=lambda x: float(x.split('h')[0].split('HL')[1]))

r1 = re.compile(".*h1")
r2 = re.compile(".*h2")
gl_series_1 = list(filter(r1.match, gl_series))
gl_series_2 = list(filter(r2.match, gl_series))
hl_series_1 = list(filter(r1.match, hl_series))
hl_series_2 = list(filter(r2.match, hl_series))

gl_series_prev_1 = ['NA'] + gl_series_1[1:]
gl_series_prev_2 = ['NA'] + gl_series_2[1:]
hl_series_prev_1 = ['NA'] + hl_series_1[1:]
hl_series_prev_2 = ['NA'] + hl_series_2[1:]

cond_list = gl_series_1 + gl_series_2 + hl_series_1 + hl_series_2
cond_prev_list = gl_series_prev_1 + gl_series_prev_2 + hl_series_prev_1 + hl_series_prev_2

is1stLast_list = ['f', 'm', 'm', 'm', 'm', 'l']*4
delt_list = ['NA', 6, 6, 12, 24, 24]*4
isTs_list = ['TRUE'] * 24
meta_df = pd.DataFrame()
meta_df['isTs'] = isTs_list
meta_df['is1stLast'] = is1stLast_list
meta_df['prevCol'] = cond_prev_list
meta_df['del.t'] = delt_list
meta_df['condName'] = cond_list

meta_df.to_csv('data/GSE111062_RAW/meta_data.tsv', sep='\t', index=False)
normalized_df.T.to_csv('data/GSE111062_RAW/expression.tsv', sep='\t')
df.to_csv('data/GSE111062_RAW/expression.tsv', sep='\t')

IndexError: list index out of range

In [11]:
df = pd.read_csv('data/GSE158898/GSE158898_genes.FPKM.annot.txt', sep='\t')

df.index = df.Gene
df = df.drop(columns=['Gene', 'gene_loci', 'gene_name', 'gene_description'])

is1stLast_list = ['f', 'm',  'm', 'l']*6
delt_list = ['NA', 12, 12, 12]*6
isTs_list = ['TRUE'] * 24
meta_df = pd.DataFrame()
cond_prev_list = []
prev_cond = 'NA'
for cond in df.columns:
    serial = int(cond.split('_')[-1])
    if (serial == 1):
        cond_prev_list.append('NA')
    else:
        cond_prev_list.append(prev_cond)
    prev_cond = cond

meta_df = pd.DataFrame()
meta_df['isTs'] = isTs_list
meta_df['is1stLast'] = is1stLast_list
meta_df['prevCol'] = cond_prev_list
meta_df['del.t'] = delt_list
meta_df['condName'] = df.columns

meta_df.to_csv('data/GSE158898/meta_data.tsv', sep='\t', index=False)

df.to_csv('data/GSE158898/expression.tsv', sep='\t')

In [34]:
raw_file_dir = 'data/GSE97500/raw_counts/'
onlyfiles = [f for f in listdir(raw_file_dir) if isfile(join(raw_file_dir, f))]
cond_names_df = pd.read_csv('data/GSE97500/series_names.tsv', sep='\t', index_col=0)

df_list = []
for raw_file in onlyfiles:
    raw_file_path = raw_file_dir + raw_file
    series_name = raw_file.split('.txt')[0].split('_')[0]
    series_name = cond_names_df['cond_name'][series_name]
    df = pd.read_csv(raw_file_path, compression='gzip', sep='\t', index_col=0, names=[series_name])
    df_list.append(df)

df = pd.concat(df_list, axis=1)
sorted_columns = sorted(df.columns, key=lambda x: (x.split('-')[1], x.split('-')[0][0], x.split('-')[0][-1], int(re.search(r'\d+', x).group())))
df = df[sorted_columns]

In [28]:
df.to_csv('data/GSE97500/expression_all.tsv', sep='\t')

In [35]:
df  = df.drop(columns=['R0-1', 'R0-2', 'R0-3', 'S0-1', 'S0-2', 'S0-3'])
df.to_csv('data/GSE97500/expression.tsv', sep='\t')

In [41]:
is1stLast_list = ['f', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'l']*12
delt_list = ['NA', 5, 5, 5, 10, 15, 15, 39, 30]*12
isTs_list = ['TRUE'] * 108
meta_df = pd.DataFrame()
cond_prev_list = []
prev_cond = 'NA'
for cond in df.columns:
    time = int(re.search(r'\d+', cond).group())
    if (time == 5):
        cond_prev_list.append('NA')
    else:
        cond_prev_list.append(prev_cond)
    prev_cond = cond

meta_df = pd.DataFrame()
meta_df['isTs'] = isTs_list
meta_df['is1stLast'] = is1stLast_list
meta_df['prevCol'] = cond_prev_list
meta_df['del.t'] = delt_list
meta_df['condName'] = df.columns

In [43]:
meta_df.to_csv('data/GSE97500/meta_data.tsv', sep='\t', index=False)
